In [1]:
cd /Users/akrim/Documents/Data Science/Stage Seoul

/Users/akrim/Documents/Data Science/Stage Seoul


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from sklearn import preprocessing
from pandas import datetime
from keras.optimizers import Adam
#import quandl
from sklearn.externals import joblib
#import backtest as twp #This is for the return and other things
import random, timeit
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline
import sys


def obv(df, n, col, vol):
    """Calculate On-Balance Volume for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    i = 0
    OBV = [0]
    while i < (df.shape[0]-1):
        if df.loc[df.index[i + 1], col] - df.loc[df.index[i], col] > 0:
            OBV.append(df.loc[df.index[i + 1], vol])
        if df.loc[df.index[i + 1], col] - df.loc[df.index[i], col] == 0:
            OBV.append(0)
        if df.loc[df.index[i + 1], col] - df.loc[df.index[i], col] < 0:
            OBV.append(-df.loc[df.index[i + 1], vol])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean(), name='OBV_' + str(n))
    return OBV_ma

def RSI(df, n, high, low): 
    """Calculate Relative Strength Index for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    i = 0  
    UpI = [0]  
    DoI = [0]  
    while i + 1 <= (data.shape[0]-1):  
        UpMove = df[high][i+1] - df[high][i]  
        DoMove = df[low][i] - df[low][i+1]
        if UpMove > DoMove and UpMove > 0:  
            UpD = UpMove  
        else: UpD = 0  
        UpI.append(UpD)  
        if DoMove > UpMove and DoMove > 0:  
            DoD = DoMove  
        else: DoD = 0  
        DoI.append(DoD)  
        i = i + 1  
    UpI = pd.Series(UpI)  
    DoI = pd.Series(DoI)  
    #print(UpI)
    PosDI = pd.Series(UpI.ewm(span = n, min_periods = n - 1).mean())  
    NegDI = pd.Series(DoI.ewm(span = n, min_periods = n - 1).mean())  
    RSI = pd.Series(PosDI / (PosDI + NegDI), name = 'RSI_' + str(n))  
    #df = df.join(RSI)  
    return RSI

#MACD, MACD Signal and MACD difference  
def MACD(df, n_fast, n_slow, col):  
    EMAfast = pd.Series(df[col].ewm(span = n_fast, min_periods = n_slow - 1,adjust=False).mean())  
    EMAslow = pd.Series(df[col].ewm(span = n_slow, min_periods = n_slow - 1,adjust=False).mean())  
    MACD = pd.Series(EMAfast - EMAslow, name = 'MACD_' + str(n_fast) + '_' + str(n_slow))  
    MACDsign = pd.Series(MACD.ewm(span = 9, min_periods = 8,adjust=False).mean(), 
                         name = 'MACDsign_' + str(n_fast) + '_' + str(n_slow))  
    MACDdiff = pd.Series(MACD - MACDsign, name = 'MACDdiff_' + str(n_fast) + '_' + str(n_slow))  
    #df = df.join(MACD)  
    #df = df.join(MACDsign)  
    #df = df.join(MACDdiff)  
    return MACD



def tradeBracket(price,entryBar,upper=None, lower=None, timeout=None):
    '''
    trade a  bracket on price series, return price delta and exit bar #
    Input
    ------
        price : numpy array of price values
        entryBar: entry bar number, *determines entry price*
        upper : high stop
        lower : low stop
        timeout : max number of periods to hold

    Returns exit price  and number of bars held

    '''
    assert isinstance(price, np.ndarray) , 'price must be a numpy array'
    
    
    # create list of exit indices and add max trade duration. Exits are relative to entry bar
    if timeout: # set trade length to timeout or series length
        exits = [min(timeout,len(price)-entryBar-1)]
    else:
        exits = [len(price)-entryBar-1] 
        
    p = price[entryBar:entryBar+exits[0]+1] # subseries of price
    
    # extend exits list with conditional exits
    # check upper bracket
    if upper:
        assert upper>p[0] , 'Upper bracket must be higher than entry price '
        idx = np.where(p>upper)[0] # find where price is higher than the upper bracket
        if idx.any(): 
            exits.append(idx[0]) # append first occurence
    # same for lower bracket
    if lower:
        assert lower<p[0] , 'Lower bracket must be lower than entry price '
        idx = np.where(p<lower)[0]
        if idx.any(): 
            exits.append(idx[0]) 
   
    
    exitBar = min(exits) # choose first exit    
  
    

    return p[exitBar], exitBar


class Backtest(object):
    """
    Backtest class, simple vectorized one. Works with pandas objects.
    """
    
    def __init__(self,price, signal, signalType='capital',initialCash = 1000, roundShares=True):
        """
        Arguments:
        
        *price*  Series with instrument price.
        *signal* Series with capital to invest (long+,short-) or number of shares. 
        *signalType* capital to bet or number of shares 'capital' mode is default.
        *initialCash* starting cash. 
        *roundShares* round off number of shares to integers
        
        """
        
        #TODO: add auto rebalancing
        
        # check for correct input
        assert signalType in ['capital','shares'], "Wrong signal type provided, must be 'capital' or 'shares'"
        
        #save internal settings to a dict
        self.settings = {'signalType':signalType}
        
        # first thing to do is to clean up the signal, removing nans and duplicate entries or exits
        self.signal = signal.ffill().fillna(0)
        
        # now find dates with a trade
        tradeIdx = self.signal.diff().fillna(0) !=0 # days with trades are set to True
        if signalType == 'shares':
            self.trades = self.signal[tradeIdx] # selected rows where tradeDir changes value. trades are in Shares
        elif signalType =='capital':
            self.trades = (self.signal[tradeIdx]/price[tradeIdx])
            if roundShares:
                self.trades = self.trades.round()
        
        # now create internal data structure 
        self.data = pd.DataFrame(index=price.index , columns = ['price','shares','value','cash','pnl'])
        self.data['price'] = price
        
        self.data['shares'] = self.trades.reindex(self.data.index).ffill().fillna(0)
        self.data['value'] = self.data['shares'] * self.data['price']
       
        delta = self.data['shares'].diff() # shares bought sold
        
        self.data['cash'] = (-delta*self.data['price']).fillna(0).cumsum()+initialCash
        self.data['pnl'] = self.data['cash']+self.data['value']-initialCash
      
      
    @property
    def sharpe(self):
        ''' return annualized sharpe ratio of the pnl '''
        pnl = (self.data['pnl'].diff()).shift(-1)[self.data['shares']!=0] # use only days with position.
        return sharpe(pnl)  # need the diff here as sharpe works on daily returns.
        
    @property
    def pnl(self):
        '''easy access to pnl data column '''
        return self.data['pnl']
    
    def plotTrades(self):
        """ 
        visualise trades on the price chart 
            long entry : green triangle up
            short entry : red triangle down
            exit : black circle
        """
        l = ['price']
        
        p = self.data['price']
        p.plot(style='x-')

        idx = (self.data['shares'] > 0) | (self.data['shares'] > 0).shift(1) 
        if idx.any():
            p[idx].plot(style='go')
            l.append('long')
        
        #colored line for short positions    
        idx = (self.data['shares'] < 0) | (self.data['shares'] < 0).shift(1) 
        if idx.any():
            p[idx].plot(style='ro')
            l.append('short')

        plt.xlim([p.index[0],p.index[-1]]) # show full axis
        
        plt.legend(l,loc='best')
        plt.title('trades')
        
        
class ProgressBar:
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 50
        self.__update_amount(0)

    def animate(self, iteration):
        sys.stdout.flush()
        self.update_iteration(iteration + 1)

    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])
    def __str__(self):
        return str(self.prog_bar)
    
def sharpe(pnl):
    return  np.sqrt(250)*pnl.mean()/pnl.std()


#download data from yahoo! finance
"""
data source: http://finance.yahoo.com/
"""


class YahooDailyReader():
    
    def __init__(self, symbol=None, start=None, end=None):
        import datetime, time
        self.symbol = symbol
        
        # initialize start/end dates if not provided
        if end is None:
            end = datetime.datetime.today()
        if start is None:
            start = datetime.datetime(1980,2,1)
        
        self.start = start
        self.end = end
        
        # convert dates to unix time strings
        unix_start = int(time.mktime(self.start.timetuple()))
        day_end = self.end.replace(hour=23, minute=59, second=59)
        unix_end = int(time.mktime(day_end.timetuple()))
        
        url = 'https://finance.yahoo.com/quote/{}/history?'
        url += 'period1={}&period2={}'
        url += '&filter=history'
        url += '&interval=1d'
        url += '&frequency=1d'
        self.url = url.format(self.symbol, unix_start, unix_end)
        
    def read(self):
        import requests, re, json
       
        r = requests.get(self.url)
        
        ptrn = r'root\.App\.main = (.*?);\n}\(this\)\);'
        txt = re.search(ptrn, r.text, re.DOTALL).group(1)
        jsn = json.loads(txt)
        df = pd.DataFrame(
                jsn['context']['dispatcher']['stores']
                ['HistoricalPriceStore']['prices']
                )
        df.insert(0, 'symbol', self.symbol)
        df['Date'] = pd.to_datetime(df['date'], unit='s').dt.date
        
        # drop rows that aren't prices
        df = df.dropna(subset=['close'])
        
        df = df[['Date', 'high', 'low', 'open', 'close', 
                 'volume', 'adjclose']]
        df = df.set_index('Date')
        df.index=pd.to_datetime(df.index)
        return df
    


#Initialize first state, all items are placed deterministically
def init_state(indata, test=False):
    close = indata['close'].values
    diff = np.diff(close) #Take the diff of the close values
    diff = np.insert(diff, 0, 0) #Insert 0 in the first position in order to 
    sma20 = indata.rolling(window=14) #Take the simple mean average for 15 days
    sma100 = indata.rolling(window=50)
    ema_short = indata.close.ewm(span=14, adjust=False).mean()
    ema_long = indata.close.ewm(span=50, adjust=False).mean()
    obv14 = obv(indata, 14, 'close', 'volume')
    rsi14 = RSI(data, 14, 'high', 'low')
    macd9 = MACD(data, 12, 26, 'close')

    #--- Preprocess data
    xdata = np.column_stack((close, diff, sma20.mean().close, close-sma20.mean().close, 
                             sma20.mean().close-sma100.mean().close, ema_short, close-ema_short, 
                             ema_short-ema_long,obv14.values, )) 
    xdata = np.nan_to_num(xdata) #Delete the NaN values
   
    if test == False:
        scaler = preprocessing.StandardScaler() #Standard scaler (unit variance and remove mean)
        xdata = np.expand_dims(scaler.fit_transform(xdata), axis=1) 
        joblib.dump(scaler, 'data/scaler.pkl')
   
    elif test == True:
        scaler = joblib.load('data/scaler.pkl')
        xdata = np.expand_dims(scaler.fit_transform(xdata), axis=1)
    state = xdata[0:1, 0:1, :] #Take only the  first values of the first day (i.e the first state)
    
    return state, xdata, close
                            
                            

#Take Action
def take_action(state, xdata, action, signal, time_step):
    #this should generate a list of trade signals that at evaluation time are fed to the backtester
    #the backtester should get a list of trade signals and a list of price data for the assett
    
    #make necessary adjustments to state and then return it
    time_step += 1
    
    #if the current iteration is the last state ("terminal state") then set terminal_state to 1
    if time_step + 1 == xdata.shape[0]:
        state = xdata[time_step-1:time_step, 0:1, :] #avant dernier day
        terminal_state = 1 #on arrive au dernier jour
        signal.loc[time_step] = 0 #signal loc 2012 avant dernier

        return state, time_step, signal, terminal_state

    #move the market data window one step forward
    state = xdata[time_step-1:time_step, 0:1, :] #next state
    #take action
    if action == 1:
        signal.loc[time_step] = 50 #Sell
    elif action == 2:
        signal.loc[time_step] = -50 #Buy
    else:
        signal.loc[time_step] = 0 #Hold
    #print(state)
    terminal_state = 0
    #print(signal)

    return state, time_step, signal, terminal_state


#Get Reward, the reward is returned at the end of an episode
def get_reward(new_state, time_step, action, xdata, signal, terminal_state, eval=False, epoch=0):
    reward = 0
    signal.fillna(value=0, inplace=True)

    if eval == False:
        bt =Backtest(pd.Series(data=[x for x in xdata[time_step-2:time_step]], index=signal[time_step-2:time_step].index.values), signal[time_step-2:time_step], signalType='shares')
        price_t = bt.data['price'].iloc[-1] #Get the last price p(t)
        price_t_1 = bt.data['price'].iloc[-2] #price p(t-1)
        #reward =(bt.data['shares'].iloc[-1]) * ((price_t) - price_t_1) / price_t_1 )
        shares_t = (bt.data['shares'].iloc[-1])
        reward =(price_t - price_t_1)*shares_t

    if terminal_state == 1 and eval == True:
        #save a figure of the test set
        bt = Backtest(pd.Series(data=[x for x in xdata], index=signal.index.values), signal, signalType='shares')
        reward = bt.pnl.iloc[-1]
        plt.figure(figsize=(3,4))
        bt.plotTrades()
        plt.axvline(x=400, color='black', linestyle='--')
        plt.text(250, 400, 'training data')
        plt.text(450, 400, 'test data')
        plt.suptitle(str(epoch))

    return reward







#MAIN 

data = YahooDailyReader('%5EGSPC').read() 
data = data.iloc[::-1]


START_TRAIN_DATE = '2010-01-01'
END_TRAIN_DATE = '2017-12-31'
START_TEST_DATE = '2018-01-01'
#END_TEST_DATE = '2018-03-09'

X_train = data.loc[(data.index <= pd.to_datetime(END_TRAIN_DATE)) & (data.index >= pd.to_datetime(START_TRAIN_DATE))]  #2013 rows
X_test = data.loc[(data.index >= pd.to_datetime(START_TEST_DATE))] #& (data.index <= pd.to_datetime(END_TEST_DATE))] #47 rows

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM

tsteps = 7
batch_size = 24
num_features =9
nb_actions = 3 #[buy, sell, hold]
np.random.seed(7)
model = Sequential()
model.add(LSTM(64, input_shape=(1, num_features), return_sequences=True,stateful=False))
model.add(Dropout(0.3))

model.add(LSTM(64, input_shape=(1, num_features), return_sequences=True,stateful=False))
model.add(Dropout(0.3))

model.add(LSTM(64,input_shape=(1, num_features), return_sequences=False,stateful=False))
model.add(Dropout(0.3))

model.add(Dense(32))
model.add(Dense(output_dim=nb_actions))
model.add(Activation('linear')) #linear output so we can have range of real-valued outputs


model.compile(loss='mean_squared_error', optimizer='Nadam')
print(model.summary())

import random, timeit

start_time = timeit.default_timer()
indata=X_train
episodes = 2
gamma = 0.9 #since the reward can be several time steps away, make gamma high
epsilon = 0.95
batchSize = 100
buffer = 100
replay = []
learning_progress = []
#stores tuples of (S, A, R, S')
h = 0
#signal = pd.Series(index=market_data.index)
signal = pd.Series(index=np.arange(len(indata)))

for i in range(episodes):
    if i == episodes-1: #the last epoch, use test data set
        #j=0
        state, xdata, price_data = init_state(X_test, test=True)
    else:
        state, xdata, price_data = init_state(indata)
    status = 1
    terminal_state = 0
    time_step = 14
    #while game still in progress
    while(status == 1):
        #We are in state S
        #Let's run our Q function on S to get Q values for all possible actions
        qval = model.predict(state, batch_size=1) #Returns a 3 array with value for each action = [sell, buy, Hold]

        if (random.random() < epsilon): #choose random action (explore)
            action = np.random.randint(1, 4) #assumes 3 different actions
        else: #choose best action from Q(s,a) values
            
            action = (np.argmax(qval))
            #if (i == episodes-1) & (signal_SSA['trade'][j]=='no') :
            #    action = 3
            #    j=j+1
        #Take action, observe new state S'
        new_state, time_step, signal, terminal_state = take_action(state, xdata, action, signal, time_step)
        #Observe reward (it's a value)
        reward = get_reward(new_state, time_step, action, price_data, signal, terminal_state)

        #Experience replay storage
        if (len(replay) < buffer): #if buffer not filled, add to it
            replay.append((state, action, reward, new_state)) #store our data
            #print(time_step, reward, terminal_state)
            
        else: #if buffer full, overwrite old values
            if (h < (buffer-1)):
                h += 1
            else:
                h = 0
            replay[h] = (state, action, reward, new_state)
            #randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)
            X_train = []
            y_train = []
            for memory in minibatch:  #So if it was a good choice, the reward will be positive and then the action will be keep for a future similar state.
                #Get max_Q(S',a)
                old_state, action, reward, new_state = memory
                old_qval = model.predict(old_state, batch_size=1) #3 values: [sell, buy, hold]
                newQ = model.predict(new_state, batch_size=1)  #Take the set of actions
                maxQ = np.max(newQ)
                y = np.zeros((1,nb_actions))
                y[:] = old_qval[:]
                if terminal_state == 0: #non-terminal state
                    update = (reward + (gamma * maxQ))
                else: #terminal state
                    update = reward
                y[0][action-1] = update
                #print(time_step, reward, terminal_state)
                X_train.append(old_state) #The inputs is the 7 features
                y_train.append(y.reshape(nb_actions,)) #The output of the neural network is 3
                

            X_train = np.squeeze(np.array(X_train), axis=(1))
            y_train = np.array(y_train)
            model.fit(X_train, y_train, batch_size=batchSize, epochs=1, verbose=0)
            
            state = new_state
        if terminal_state == 1: #if reached terminal state, update epoch status
            status = 0
            
        
    if epsilon > 0.1: #decrement epsilon over time
        epsilon -= (1.0/episodes)
    print(i+1)
    
    ### GIVE INFORMATION ABOUT THE RETURN / PRICE ###
    bt = Backtest(pd.Series(data=[x[0,0] for x in xdata]), signal, signalType='shares')
    bt.data['delta'] = bt.data['shares'].diff().fillna(0)
        
        #print(bt.data)
    unique, counts = np.unique(filter(lambda v: v==v, signal.values), return_counts=True)
#print(np.asarray((unique, counts)).T)
    plt.rcParams['figure.figsize']=(20,13)

    bt.plotTrades() #Plot the different trades (Hold, Buy or Sold)
    plt.show()
    
    plt.title("Evolution of the P&L")
    bt.pnl.plot() #Plot the PNL
    plt.show()

Using TensorFlow backend.
/Users/akrim/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:438: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 64)             18944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 64)             33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
__________

In [ ]:
class SSA(object):
    
    __supported_types = (pd.Series, np.ndarray, list)
    
    def __init__(self, tseries, L, save_mem=True):
        """
        Decomposes the given time series with a singular-spectrum analysis. Assumes the values of the time series are
        recorded at equal intervals.
        
        Parameters
        ----------
        tseries : The original time series, in the form of a Pandas Series, NumPy array or list. 
        L : The window length. Must be an integer 2 <= L <= N/2, where N is the length of the time series.
        save_mem : Conserve memory by not retaining the elementary matrices. Recommended for long time series with
            thousands of values. Defaults to True.
        
        Note: Even if an NumPy array or list is used for the initial time series, all time series returned will be
        in the form of a Pandas Series or DataFrame object.
        """
        
        # Tedious type-checking for the initial time series
        if not isinstance(tseries, self.__supported_types):
            raise TypeError("Unsupported time series object. Try Pandas Series, NumPy array or list.")
        
        # Checks to save us from ourselves
        self.N = len(tseries)
        if not 2 <= L <= self.N/2:
            raise ValueError("The window length must be in the interval [2, N/2].")
        
        self.L = L
        self.orig_TS = pd.Series(tseries)
        self.K = self.N - self.L + 1
        
        # Embed the time series in a trajectory matrix
        self.X = np.array([self.orig_TS.values[i:L+i] for i in range(0, self.K)]).T
        
        # Decompose the trajectory matrix
        self.U, self.Sigma, VT = np.linalg.svd(self.X)
        self.d = np.linalg.matrix_rank(self.X)
        
        self.TS_comps = np.zeros((self.N, self.d))
        
        if not save_mem:
            # Construct and save all the elementary matrices
            self.X_elem = np.array([ self.Sigma[i]*np.outer(self.U[:,i], VT[i,:]) for i in range(self.d) ])

            # Diagonally average the elementary matrices, store them as columns in array.           
            for i in range(self.d):
                X_rev = self.X_elem[i, ::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.V = VT.T
        else:
            # Reconstruct the elementary matrices without storing them
            for i in range(self.d):
                X_elem = self.Sigma[i]*np.outer(self.U[:,i], VT[i,:])
                X_rev = X_elem[::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.X_elem = "Re-run with save_mem=False to retain the elementary matrices."
            
            # The V array may also be very large under these circumstances, so we won't keep it.
            self.V = "Re-run with save_mem=False to retain the V matrix."
        
        # Calculate the w-correlation matrix.
        self.calc_wcorr()
            
    def components_to_df(self, n=0):
        """
        Returns all the time series components in a single Pandas DataFrame object.
        """
        if n > 0:
            n = min(n, self.d)
        else:
            n = self.d
        
        # Create list of columns - call them F0, F1, F2, ...
        cols = ["F{}".format(i) for i in range(n)]
        return pd.DataFrame(self.TS_comps[:, :n], columns=cols, index=self.orig_TS.index)
            
    
    def reconstruct(self, indices):
        """
        Reconstructs the time series from its elementary components, using the given indices. Returns a Pandas Series
        object with the reconstructed time series.
        
        Parameters
        ----------
        indices: An integer, list of integers or slice(n,m) object, representing the elementary components to sum.
        """
        if isinstance(indices, int): indices = [indices]
        
        ts_vals = self.TS_comps[:,indices].sum(axis=1)
        return pd.Series(ts_vals, index=self.orig_TS.index)
    
    def calc_wcorr(self):
        """
        Calculates the w-correlation matrix for the time series.
        """
             
        # Calculate the weights
        w = np.array(list(np.arange(self.L)+1) + [self.L]*(self.K-self.L-1) + list(np.arange(self.L)+1)[::-1])
        
        def w_inner(F_i, F_j):
            return w.dot(F_i*F_j)
        
        # Calculated weighted norms, ||F_i||_w, then invert.
        F_wnorms = np.array([w_inner(self.TS_comps[:,i], self.TS_comps[:,i]) for i in range(self.d)])
        F_wnorms = F_wnorms**-0.5
        
        # Calculate Wcorr.
        self.Wcorr = np.identity(self.d)
        for i in range(self.d):
            for j in range(i+1,self.d):
                self.Wcorr[i,j] = abs(w_inner(self.TS_comps[:,i], self.TS_comps[:,j]) * F_wnorms[i] * F_wnorms[j])
                self.Wcorr[j,i] = self.Wcorr[i,j]
    
    def plot_wcorr(self, min=None, max=None):
        """
        Plots the w-correlation matrix for the decomposed time series.
        """
        if min is None:
            min = 0
        if max is None:
            max = self.d
        
        if self.Wcorr is None:
            self.calc_wcorr()
        
        ax = plt.imshow(self.Wcorr)
        plt.xlabel(r"$\tilde{F}_i$")
        plt.ylabel(r"$\tilde{F}_j$")
        plt.colorbar(ax.colorbar, fraction=0.045)
        ax.colorbar.set_label("$W_{i,j}$")
        plt.clim(0,1)
        
        # For plotting purposes:
        if max == self.d:
            max_rnge = self.d-1
        else:
            max_rnge = max
        
        plt.xlim(min-0.5, max_rnge+0.5)
        plt.ylim(max_rnge+0.5, min-0.5)

In [ ]:
data_filter_pnl=bt.pnl
(data_filter_pnl.shape[0]/2)*0.7 #N/2*0.7
window = 59 # samples
accel_ssa = SSA(data_filter_pnl, window)
accel_ssa.plot_wcorr(max=30)
plt.title("W-Correlation for Walking Time Series (Zoomed)");


In [ ]:
accel_ssa.reconstruct(slice(0,4)).plot()
#accel_ssa.reconstruct(slice(12,window)).plot()
accel_ssa.orig_TS.plot( alpha=0.5)

plt.legend([r"$\tilde{F}^{\mathrm{(signal)}}$", r"$\tilde{F}^{\mathrm{(noise)}}$"])
plt.title("Signal and Noise Components of Toy Time Series, $L = 3726$")
plt.rcParams['figure.figsize']=(20,13)

plt.xlabel(r"$t$");

In [ ]:
signal_SSA=pd.DataFrame(accel_ssa.reconstruct(slice(0,4)))
signal_SSA['diff']=0
signal_SSA['diff'][:(len(signal_SSA)-1)]= np.diff(signal_SSA[0].values)
signal_SSA['diff'][(len(signal_SSA))-1]=signal_SSA['diff'][(len(signal_SSA))-2]
signal_SSA['trade'] = np.where(signal_SSA['diff']>=0, 'yes', 'no')
